In [148]:
import cplex
from cplex.exceptions import CplexError
import sys
import pandas as pd
import numpy as np
import docplex

In [149]:
def pprint(constraints: list, names: list):
    for c in constraints:
        _c = [(("-"if _i == -1 else "" if _i == 1 else (str(_i if isinstance(_i, int) else round(_i,1)))) + n) for _i, n in zip(c[0], names) if _i != 0]
        print(c[3] + ": ([" + " ".join(_c) + "]  " + c[1] + "  " + str(c[2] if isinstance(c[2], int) else round(c[2],1)) + ")") 

In [150]:
xls = pd.ExcelFile('logistic.xlsx')

vehicles = ["1","2","3","4","5"]                        # V
customers = ["1","2","3","4","5","6","7","8","9"]       # C

num_vehicles = 2
num_customers = 2
locations = ["O"] + customers[:num_customers] + ["X"]   # C': gồm các customers và Depot suất phát + Depot kết thúc

# quy ước: 
#    + (i,j) là cạnh (edge) thuộc tập E
#    + i,j là 2 customer liền kề trên 1 route


In [151]:
my_names = ["x"+str(i)+str(j) + str(k) for k in vehicles[:num_vehicles] for i in (["O"] + customers[:num_customers]) for j in (customers[:num_customers] + ["X"])]
num_names = len(my_names)
print(num_names)
print("Names ", my_names)
def display(constraints):
    pprint(constraints, my_names)

18
Names  ['xO11', 'xO21', 'xOX1', 'x111', 'x121', 'x1X1', 'x211', 'x221', 'x2X1', 'xO12', 'xO22', 'xOX2', 'x112', 'x122', 'x1X2', 'x212', 'x222', 'x2X2']


In [152]:
cost_matrix = pd.read_excel(xls, 'Cost Matrix')
__v = lambda n: 0 if n == "O" or n == "X" else int(n)
my_obj = [int(cost_matrix.iat[__v(n[1]),__v(n[2]) + 1]) for n in my_names]

print("Object ", my_obj)

Object  [18, 30, 0, 0, 23, 18, 23, 0, 30, 18, 30, 0, 0, 23, 18, 23, 0, 30]


In [153]:
capacity_matrix   = pd.read_excel(xls, 'Capicity')
vehicles_capacity = [capacity_matrix.iat[i,1] for i in range(0, num_vehicles)]
vehicles_capacity

[65, 70]

In [154]:
demand_matrix = pd.read_excel(xls, 'Demand Matrix')
demand = {customers[i]: demand_matrix.iat[i,1] for i in range(0, num_customers)}
demand

{'1': 32, '2': 45}

In [155]:
lower_bounds = [0]*num_names
upper_bounds = [1]*num_names

print("Lb ", lower_bounds)
print("Rb ", upper_bounds)

Lb  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Rb  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [156]:
# CONSTRAINTS_2: 
# # Một cạnh bất kỳ (i->j) chỉ tổn tại trên duy nhất 1 Vehicle. Nhưng i CHỈ LÀ customers, j là locations
constraint_2 = []
_i = 0
for i in customers[:num_customers]:
    con_ = [(1 if name[1] != name[2]  and name[1] == i else 0) for name in my_names]
    if len([c for c in con_  if c != 0]) > 0:
        _i += 1
        constraint_2.append((con_,"E", 1, "C2" + str(_i)))
    
display(constraint_2)


C21: ([x121 x1X1 x122 x1X2]  E  1)
C22: ([x211 x2X1 x212 x2X2]  E  1)


In [157]:
# CONSTRAINTS_3: 
# # Lượng hàng chở đển Customer của 1 Vehicle phải nhỏ hơn bằng Capacity. Nhưng i CHỈ LÀ customers, j là locations
constraint_3 = []
val = lambda v, c, n1, n2, n3: 0 if n3 != v else -1*int(demand[n1]) if n1 != "O" and n1 != n2 else c if n1 == "O" and n2 != "O" else 0
# val = lambda v, c, n1, n2, n3: 0 if n3 != v else int(demand[n1]) if n1 != "O" and n1 != n2 else 0
_i = 0
for veh, cap in zip(vehicles[0:num_vehicles],vehicles_capacity[0:num_vehicles]): 
    _i += 1
    constraint_3.append(([val(veh, float(cap), name[1], name[2], name[3]) for name in my_names],"G", 0, "C3" + str(_i)))


display(constraint_3)

C31: ([65.0xO11 65.0xO21 65.0xOX1 -32x121 -32x1X1 -45x211 -45x2X1]  G  0)
C32: ([70.0xO12 70.0xO22 70.0xOX2 -32x122 -32x1X2 -45x212 -45x2X2]  G  0)


In [158]:
# CONSTRAINTS_4: 
# # Với mỗi Vehicle, khi suất phát từ Depot, nó chỉ có thể đến duy nhất 1 Customer
constraint_4 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    _i += 1
    constraint_4.append(([(1 if (name[1] == "O" and name[3] == k) else 0) for name in my_names],"E", 1, "C4" + str(_i)))
    
display(constraint_4)

C41: ([xO11 xO21 xOX1]  E  1)
C42: ([xO12 xO22 xOX2]  E  1)


In [159]:
# CONSTRAINTS_5: 
# # Trên route của Vehicle k có cạnh (i->j) : tổng tất cả cạnh đầu 'ra' của i BẰNG tổng tất cả cạnh đầu 'vào' của j
constraint_5 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    for p in customers[:num_customers]: # i
        _i += 1
        foo = lambda k,p,n: 0 if n[3] != k or n[2] == n[1] else 1 if n[1] == p else -1 if n[2] == p else 0                
        constraint_5.append(([foo(k, p, name) for name in my_names],"E", 0, "C5" + str(_i)))

display(constraint_5) 

C51: ([-xO11 x121 x1X1 -x211]  E  0)
C52: ([-xO21 -x121 x211 x2X1]  E  0)
C53: ([-xO12 x122 x1X2 -x212]  E  0)
C54: ([-xO22 -x122 x212 x2X2]  E  0)


In [160]:
# CONSTRAINTS_6: 
# # Route của Vehicle k phải luôn trở về Depot (n+1), tức là luôn tồn tại điểm i trên route của k sao cho nó đi đến n+1
# # trong đó n+1 là Depot kết thúc
constraint_6 = []
_i = 0
for veh in vehicles[0:num_vehicles]: 
    _i += 1
    constraint_6.append(([(1 if (name[3] == veh and name[2] == "X") else 0) for name in my_names],"E", 1, "C6" + str(_i)))
    
display(constraint_6)


C61: ([xOX1 x1X1 x2X1]  E  1)
C62: ([xOX2 x1X2 x2X2]  E  1)


In [161]:
# CONSTRAINTS_X: vehicles ko đc đứng tại chỗ
constraint_x = []
_i = 0

_i += 1
constraint_x.append(([(1 if name[1] == name[2] else 0) for name in my_names],"E", 0, "CX" + str(_i)))

display(constraint_x)


CX1: ([x111 x221 x112 x222]  E  0)


In [162]:
# CONSTRAINTS_Y: vehicles ko đc quay ngược lại
constraint_y = []
_i = 0

for i in customers[:num_customers]:
    for j in customers[:num_customers]:
        if i != j:
            _i += 1
            constraint_y.append(([(1 if ((name[1] == i and j == name[2]) or (name[1] == j and i == name[2])) else 0) for name in my_names],"L", 1, "CX" + str(_i)))

display(constraint_y)


CX1: ([x121 x211 x122 x212]  L  1)
CX2: ([x121 x211 x122 x212]  L  1)


In [163]:
def foo(n : int, a : list, o : list):
    temp = {}
    tempa = []
    for ai in a:
        _i =  np.prod([ord(c) + 99 for c in ai])
        if _i not in temp:            
            temp[_i] = ai
            tempa.append(ai)
            
    a = tempa
    arr = []
    if n > 1:
        for ai in a:
            for oi in o:
                if oi not in ai:
                    arr.append(ai + oi)
        return foo(n-1, arr, o)
    else:
        return a

def get_path(n: int, a: list) -> list:
    return foo(n,a,a)


In [164]:
# CONSTRAINT_Z 
constraint_z = []
_i = 0

for r in range(3,num_customers + 1):
    for p in get_path(r, customers[:num_customers]):
#         p = "134"
#         r = 3
        _i += 1
        constraint_z.append(([1 if name[1] in p and name[2] in p and name[1] != name[2] else 0 for name in my_names], "L", r-1, "CZ" + str(_i)))

display(constraint_z)


In [165]:
# TIME
time_names = ["s"+str(i) + str(k) for i in (["O"] + customers[:num_customers] + ["X"]) for k in vehicles[:num_vehicles]]
time_names += ["e"+str(i) + str(k) for i in (["O"] + customers[:num_customers] + ["X"]) for k in vehicles[:num_vehicles]]

full_names = my_names + time_names
print(time_names)
def displayt(constraints):
    pprint(constraints, time_names)

def displayful(constraints):
    pprint(constraints, full_names)

['sO1', 'sO2', 's11', 's12', 's21', 's22', 'sX1', 'sX2', 'eO1', 'eO2', 'e11', 'e12', 'e21', 'e22', 'eX1', 'eX2']


In [166]:
# constraints time 1 s0k = 0 ủa cho thành tổng tất cả là 0 dc ko 
constraints_time_1 = []
constraints_time_1.append(([1 if name[0] == "s" and name[1] == "O" else 0 for name in full_names], "E", 0 , "CT1" ))
    
displayful(constraints_time_1)


CT1: ([sO1 sO2]  E  0)


In [167]:
# Time window 
start_time = pd.read_excel(xls, 'Time Window')['Time Start']
print(start_time)
start_time= {loc: float(start_time.iat[i]) for i, loc in zip(range(0,num_customers + 1), ["O"] + customers[:num_customers])}

end_time = pd.read_excel(xls, 'Time Window')['Time End']
end_time = {loc: float(end_time.iat[i]) for i, loc in zip(range(0,num_customers + 1), ["O"] + customers[:num_customers])} 

print(start_time)
print(end_time)


0     0.0
1     6.0
2     0.0
3     1.0
4     1.0
5     1.0
6     1.0
7    15.0
8     4.5
9     6.0
Name: Time Start, dtype: float64
{'O': 0.0, '1': 6.0, '2': 0.0}
{'O': 5.0, '1': 13.0, '2': 15.0}


In [168]:
# Time matrix
time_matrix = pd.read_excel(xls, 'Time Matrix')

timesdf = time_matrix.iloc[0:num_customers + 2,1:num_customers + 2].values

times = {}
for _i, i in enumerate(["O"] + customers[:num_customers]):
    for _j, j in enumerate(["O"] + customers[:num_customers]):
        key = i+j
        val = timesdf[_i][_j]
        if not key in times:
            times[key]=val
print(timesdf)    
times

[[0.  3.  4.5]
 [3.  0.  2.7]
 [4.5 2.7 0. ]
 [2.  3.2 2.1]]


{'OO': 0.0,
 'O1': 3.0,
 'O2': 4.5,
 '1O': 3.0,
 '11': 0.0,
 '12': 2.7,
 '2O': 4.5,
 '21': 2.7,
 '22': 0.0}

In [169]:
# constraints time 2 for j in() xijk*e_i <= s_ik  (xi1k_+xi2k+...+xink)*l_i <= sik 
full_names = my_names + time_names
constraints_time_2 = []
_i=0
for i in (["O"] + customers[:num_customers]):
    for k in vehicles[:num_vehicles]:
        _i+=1
        constraints_time_2.append(([(-1 if name[1] == i and name[2] == k else 0) if name[0] == "s" \
                                    else start_time[i] if (name[0] == "x" and name[1] == i and name[3] == k) else 0 for name in full_names], "L", 0 , "CT2" +str(_i)))
displayful(constraints_time_2)

CT21: ([-sO1]  L  0)
CT22: ([-sO2]  L  0)
CT23: ([6.0x111 6.0x121 6.0x1X1 -s11]  L  0)
CT24: ([6.0x112 6.0x122 6.0x1X2 -s12]  L  0)
CT25: ([-s21]  L  0)
CT26: ([-s22]  L  0)


In [170]:
# constraints_time_3 s_ik <= l_i*sum(xijk)
constraints_time_3 = []
_i=0

for i in (["O"] + customers[:num_customers]):
    for k in vehicles[:num_vehicles]:
        _i+=1
        constraints_time_3.append(([(-1 if name[1] == i and name[2] == k else 0) if name[0] == "s" \
                                    else end_time[i] if (name[0] == "x" and name[1] == i and name[3] == k) else 0 for name in full_names], "G", 0 , "CT3" +str(_i)))
displayful(constraints_time_3)


CT31: ([5.0xO11 5.0xO21 5.0xOX1 -sO1]  G  0)
CT32: ([5.0xO12 5.0xO22 5.0xOX2 -sO2]  G  0)
CT33: ([13.0x111 13.0x121 13.0x1X1 -s11]  G  0)
CT34: ([13.0x112 13.0x122 13.0x1X2 -s12]  G  0)
CT35: ([15.0x211 15.0x221 15.0x2X1 -s21]  G  0)
CT36: ([15.0x212 15.0x222 15.0x2X2 -s22]  G  0)


In [176]:
# constraints time 4
constraints_time_4 = []
_i=0
    
for k in vehicles[:num_vehicles]:
    for i in ["O"] + customers[:num_customers]:
        for j in customers[:num_customers] + ["X"]:
            if i==j or j == "X":
                continue            
            _m = max(end_time[i] + times[i+("O" if j == "X" else j)] - start_time[i], 0)
            
            obj = [1 if name[1] == i and name[0] == "e" and name[2] == k else 
                    -1 if name[1] == j and name[0] == "s" and name[2] == k else
                   _m if name[0] == "x" and name[1] == i and name[2] == j and name[3] == k else 0
                   for name in full_names if i != j]
        
            rhs = _m - times[i+("O" if j == "X" else j)]
            
            if len([o for o in obj if o != 0]) > 0:
                _i+=1
                constraints_time_4.append((obj, "L", rhs , "CT3" +str(_i)))
                
                
displayful(constraints_time_4)

CT31: ([8.0xO11 -s11 eO1]  L  5.0)
CT32: ([9.5xO21 -s21 eO1]  L  5.0)
CT33: ([9.7x121 -s21 e11]  L  7.0)
CT34: ([17.7x211 -s11 e21]  L  15.0)
CT35: ([8.0xO12 -s12 eO2]  L  5.0)
CT36: ([9.5xO22 -s22 eO2]  L  5.0)
CT37: ([9.7x122 -s22 e12]  L  7.0)
CT38: ([17.7x212 -s12 e22]  L  15.0)


In [172]:
# CONSTRAINTS TIME 5: eik - sik > 0
constraints_time_5 = []
_i=0

for i in (["O"] + customers[:num_customers]):
    for k in vehicles[:num_vehicles]:
        _i+=1
        constraints_time_5.append(([-1 if name[1] == i and name[2] == k and name[0] == "s" else 
                                    1 if (name[0] == "e" and name[1] == i and name[2] == k) else 0
                                    for name in full_names], "G", 0 , "CT5" +str(_i)))
displayful(constraints_time_5)


CT51: ([-sO1 eO1]  G  0)
CT52: ([-sO2 eO2]  G  0)
CT53: ([-s11 e11]  G  0)
CT54: ([-s12 e12]  G  0)
CT55: ([-s21 e21]  G  0)
CT56: ([-s22 e22]  G  0)


In [173]:
# Merge segment constraints
constraints_group = []
constraints_group += constraint_2 
constraints_group += constraint_3 
constraints_group += constraint_4 
constraints_group += constraint_5 
constraints_group += constraint_6
constraints_group += constraint_x
constraints_group += constraint_y
constraints_group += constraint_z
# constraints_group += constraints_time_1
constraints_group += constraints_time_2
constraints_group += constraints_time_3
constraints_group += constraints_time_4
constraints_group += constraints_time_5


# Convert for cplex
constraints_ = []
constraint_senses = []
rhs = []
constraint_names = []
for i in constraints_group:
    constraints_.append([full_names if i[3][:2] == "CT" else my_names, i[0]])
    constraint_senses.append(i[1])
    rhs.append(i[2])
    constraint_names.append(i[3])
    
# print(constraints_)
# print(constraint_senses)
# print(rhs)

In [174]:
# Create Cplex Problems

myProblem = None

try:
    myProblem = cplex.Cplex()     
except CplexError:
    print ("exc")
# print(len(my_obj), len(my_names))
myProblem.variables.add(obj = my_obj,
                      lb = lower_bounds,
                      ub = upper_bounds,
                      names = my_names)

myProblem.variables.add(names= time_names, 
                        types = ["C"]*len(time_names),
                        ub = [24]*len(time_names),
                        lb = [-24]*len(time_names)) 


myProblem.objective.set_sense(myProblem.objective.sense.minimize)

for i in range(0, num_names):
    myProblem.variables.set_types(i,myProblem.variables.type.binary)

myProblem.linear_constraints.add(lin_expr = constraints_,
                                 senses = constraint_senses,
                                 rhs = rhs,
                                 names = constraint_names)

# ------------------------------------------------------------------------------------
print(myProblem.variables.get_names())
print(myProblem.variables.get_types())
print(myProblem.variables.get_upper_bounds())
print(myProblem.variables.get_lower_bounds())


['xO11', 'xO21', 'xOX1', 'x111', 'x121', 'x1X1', 'x211', 'x221', 'x2X1', 'xO12', 'xO22', 'xOX2', 'x112', 'x122', 'x1X2', 'x212', 'x222', 'x2X2', 'sO1', 'sO2', 's11', 's12', 's21', 's22', 'sX1', 'sX2', 'eO1', 'eO2', 'e11', 'e12', 'e21', 'e22', 'eX1', 'eX2']
['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0]


In [175]:
# Find the solutions
try:
    myProblem.solve()
    
    print(myProblem.get_stats())
    values=myProblem.solution.get_values()
    names=myProblem.variables.get_names()
    
    print("-"*30)
    for v, n in zip(values, names):
        if v > 0:
            print(n, " - ",v if isinstance(v, int) else round(v,1))
except cplex.exceptions.errors.CplexSolverError:
    print("-"*30,"\nNo solution")
    


Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 5 times.
MIP Presolve eliminated 24 rows and 17 columns.
MIP Presolve modified 55 coefficients.
Aggregator did 17 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.21 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.22 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.22 ticks)
Problem name         : 
Objective sense      : Minimize
Variables            :      34  [Box: 16,  Binary: 18]
Objective nonzeros   :      12
Linear constraints   :      41  [Less: 16,  Greater: 14,  Equal: 11]
  Nonzeros           :     134
  RHS nonzeros       :      16

Variables            : Min LB: -24.00000        Max UB: 24.00000     